<a href="https://colab.research.google.com/github/buzacott/frontiers_ecohydrology/blob/main/frontiers_ecohydrology_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frontiers in Hydrology: Ecohydrology data preparation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from calendar import monthrange

# Change default plot size
plt.rcParams["figure.figsize"] = (9,6)

In [2]:
# Datetime parser for variable dt lengths
def flx_date_parser(date):
    if(len(date)==4):
        pattern = '%Y'
    if(len(date)==6):
        pattern = '%Y%m'
    if(len(date)==8):
        pattern = '%Y%m%d'
    if(len(date)==12):
        pattern = '%Y%m%d%H%M'
    
    return dt.datetime.strptime(date, pattern)

## Load data from GitHub

The flux data is currently a combined file that contains the monthly FLUXNET2015 Tier 1 data. Can switch to daily but for experimenting monthly is fine.

In [3]:
flx = pd.read_csv('https://raw.githubusercontent.com/buzacott/frontiers_ecohydrology/main/data/fluxnet2015_combined_monthly_data.csv')
flx['TIMESTAMP'] = pd.to_datetime(flx['TIMESTAMP'])
# Split site ID into country and site name
flx[['Country', 'Site']] = flx['SITE_ID'].str.split('-', expand=True)
flx.head()

,TIMESTAMP,SITE_ID,TA_F,SW_IN_POT,SW_IN_F,LW_IN_F,VPD_F,PA_F,P_F,WS_F,...,H_RANDUNC,NEE_VUT_REF,RECO_NT_VUT_REF,GPP_NT_VUT_REF,PPFD_DIF,SW_DIF,LW_OUT,ET_F_MDS,Country,Site
0,2010-01-01,SN-Dhr,29.467,342.963,11.044,330.718,23.842,100.766,0.000,2.996,...,NaN,-7.40131,6.99884,14.3139,NaN,NaN,NaN,4.123013,SN,Dhr
1,2010-02-01,SN-Dhr,28.795,380.174,12.317,353.484,23.794,100.602,0.000,2.906,...,NaN,-7.42792,6.86155,14.2089,NaN,NaN,NaN,4.119353,SN,Dhr
2,2010-03-01,SN-Dhr,28.019,419.192,14.196,358.876,23.368,100.492,0.000,3.075,...,NaN,-7.40865,6.70299,14.0398,NaN,NaN,NaN,4.117739,SN,Dhr
3,2010-04-01,SN-Dhr,27.762,443.462,14.655,385.542,23.424,100.388,0.052,3.335,...,NaN,-7.41212,6.65065,13.9931,NaN,NaN,NaN,4.117146,SN,Dhr
4,2010-05-01,SN-Dhr,27.221,449.791,15.075,402.832,23.192,100.414,0.000,3.080,...,NaN,-7.40131,6.54057,13.8779,NaN,NaN,NaN,4.116474,SN,Dhr


BADM (Biological, Ancillary, Disturbance, and Metadata) contain Site General Info, Canopy Height, and Variable Info

In [4]:
badm = pd.read_csv('https://raw.githubusercontent.com/buzacott/frontiers_ecohydrology/main/data/fluxnet2015_badm_monthly.csv')
badm.head()

,SITE_ID,GROUP_ID,VARIABLE_GROUP,VARIABLE,DATAVALUE
0,AR-SLu,14214,GRP_CLIM_AVG,MAP,400
1,AR-SLu,27000554,GRP_COUNTRY,COUNTRY,Argentina
2,AR-SLu,81739,GRP_DOI,DOI,10.18140/FLX/1440191
3,AR-SLu,81739,GRP_DOI,DOI_DATAPRODUCT,FLUXNET2015
4,AR-SLu,30007072,GRP_DOI_CONTRIBUTOR,DOI_CONTRIBUTOR_NAME,Alfredo Garcia


In [5]:
badm.VARIABLE_GROUP.unique()

array(['GRP_CLIM_AVG', 'GRP_COUNTRY', 'GRP_DOI', 'GRP_DOI_CONTRIBUTOR',
       'GRP_DOI_ORGANIZATION', 'GRP_HEADER', 'GRP_IGBP', 'GRP_LOCATION',
       'GRP_NETWORK', 'GRP_REFERENCE_PAPER', 'GRP_TEAM_MEMBER',
       'GRP_URL_FLUXNET', 'GRP_UTC_OFFSET', 'GRP_HEIGHTC', 'GRP_VAR_INFO',
       'GRP_DOM_DIST_MGMT', 'GRP_FLUX_MEASUREMENTS', 'GRP_RESEARCH_TOPIC',
       'GRP_SITE_CHAR', 'GRP_SITE_DESC', 'GRP_SITE_FUNDING',
       'GRP_TOWER_POWER', 'GRP_TOWER_TYPE', 'GRP_URL',
       'GRP_ACKNOWLEDGEMENT', 'GRP_LAND_OWNERSHIP'], dtype=object)

Extract key variables

In [6]:
# International Geosphere-Biosphere Programme land use code
# https://fluxnet.org/data/badm-data-templates/igbp-classification/
landuse = badm.query('VARIABLE_GROUP == "GRP_IGBP"') \
              .pivot(index=['SITE_ID', 'GROUP_ID'],
                     columns = 'VARIABLE',
                     values = 'DATAVALUE') \
              .reset_index()
# Lat/lon
location = badm.query('VARIABLE_GROUP == "GRP_LOCATION"') \
               .pivot(index=['SITE_ID', 'GROUP_ID'],
                      columns = 'VARIABLE',
                      values = 'DATAVALUE') \
               .reset_index()
# Some duplicates, some sites move. Filter them for now
location.drop_duplicates(subset='SITE_ID', inplace=True)

# Canopy height
canopy_height = badm.query('VARIABLE_GROUP == "GRP_HEIGHTC"') \
                    .pivot(index=['SITE_ID', 'GROUP_ID'],
                           columns = 'VARIABLE',
                           values = 'DATAVALUE') \
                    .reset_index() \
                    .rename({'HEIGHTC_DATE': 'TIMESTAMP'}, axis=1) \
                    .infer_objects()
canopy_height['TIMESTAMP'] = [flx_date_parser(d) for d in canopy_height['TIMESTAMP']]

In [7]:
# Update dtypes, infer_objects() does not work
def update_dtypes(df):
  cols = df.columns
  for c in cols:
      if c is not 'TIMESTAMP':
        try:
            df[c] = pd.to_numeric(df[c])
        except:
            pass
  return df

location = update_dtypes(location)
canopy_height = update_dtypes(canopy_height)

In [8]:
# Create a canopy height value for each timestep
# Some site have many values, others only 1
canopy_height_full = flx[['TIMESTAMP', 'SITE_ID']] \
  .merge(canopy_height[['TIMESTAMP', 'SITE_ID', 'HEIGHTC']],
         how='left')
canopy_height_full['HEIGHTC'] = canopy_height_full.groupby('SITE_ID', sort=False)['HEIGHTC'].apply(lambda x: x.ffill().bfill())

Add in NDVI that has been extracted from MOD13Q1.061 via GEE

In [9]:
ndvi = pd.read_csv('https://raw.githubusercontent.com/buzacott/frontiers_ecohydrology/main/data/fluxnet2015_ndvi_monthly.csv')
ndvi['TIMESTAMP'] = pd.to_datetime(ndvi['TIMESTAMP'])
ndvi.head()

,SITE_ID,TIMESTAMP,NDVI
0,AR-SLu,2000-02-01,0.57160
1,AR-SLu,2000-03-01,0.59320
2,AR-SLu,2000-04-01,0.57055
3,AR-SLu,2000-05-01,0.50160
4,AR-SLu,2000-06-01,0.44330


### Merge data frames together

In [10]:
# Join ndvi into df
df = flx.merge(ndvi,
               how='left',
               left_on=['TIMESTAMP', 'SITE_ID'],
               right_on=['TIMESTAMP', 'SITE_ID']) \
       .merge(landuse[['SITE_ID', 'IGBP']],
              how='left',
              left_on='SITE_ID',
              right_on='SITE_ID') \
       .merge(location[['SITE_ID', 'LOCATION_LAT', 'LOCATION_LONG']],
              left_on='SITE_ID',
              right_on='SITE_ID') \
       .merge(canopy_height_full,
              left_on=['TIMESTAMP', 'SITE_ID'],
              right_on=['TIMESTAMP', 'SITE_ID'])

df.set_index('TIMESTAMP', inplace=True)
df

,SITE_ID,TA_F,SW_IN_POT,SW_IN_F,LW_IN_F,VPD_F,PA_F,P_F,WS_F,USTAR,...,SW_DIF,LW_OUT,ET_F_MDS,Country,Site,NDVI,IGBP,LOCATION_LAT,LOCATION_LONG,HEIGHTC
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2010-01-01,SN-Dhr,29.467,342.963,11.044,330.718,23.842,100.766,0.000,2.996,NaN,...,NaN,NaN,4.123013,SN,Dhr,0.20875,SAV,15.40278,-15.43222,1.58
2010-02-01,SN-Dhr,28.795,380.174,12.317,353.484,23.794,100.602,0.000,2.906,NaN,...,NaN,NaN,4.119353,SN,Dhr,0.21285,SAV,15.40278,-15.43222,1.58
2010-03-01,SN-Dhr,28.019,419.192,14.196,358.876,23.368,100.492,0.000,3.075,NaN,...,NaN,NaN,4.117739,SN,Dhr,0.18560,SAV,15.40278,-15.43222,1.58
2010-04-01,SN-Dhr,27.762,443.462,14.655,385.542,23.424,100.388,0.052,3.335,NaN,...,NaN,NaN,4.117146,SN,Dhr,0.18870,SAV,15.40278,-15.43222,1.58
2010-05-01,SN-Dhr,27.221,449.791,15.075,402.832,23.192,100.414,0.000,3.080,NaN,...,NaN,NaN,4.116474,SN,Dhr,0.17735,SAV,15.40278,-15.43222,1.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-08-01,BR-Sa3,25.605,406.055,181.901,411.642,9.201,98.732,2.837,1.804,NaN,...,NaN,NaN,2.163218,BR,Sa3,0.86445,EBF,-3.01803,-54.97144,25.00
2004-09-01,BR-Sa3,26.004,429.912,204.215,412.823,9.329,98.587,3.696,2.018,NaN,...,NaN,NaN,2.172189,BR,Sa3,NaN,EBF,-3.01803,-54.97144,25.00
2004-10-01,BR-Sa3,26.708,440.106,217.381,416.859,10.586,98.473,2.546,2.127,NaN,...,NaN,NaN,2.177463,BR,Sa3,0.73560,EBF,-3.01803,-54.97144,25.00


In [11]:
df.to_csv('fluxnet2015_data_for_students.csv')